<h1 style='color:orange'>Langraph</h1>

# Purpose:Explain key concepts of langgraph
I am going to use Langgraph for implementing a simple Authentication and a Question/Answer with ollama chat

# 1) Authentication

In [32]:
from typing import TypedDict,Optional,Tuple
from langgraph.graph import StateGraph
from langgraph.graph import END

**State:**
It is like a dictionnary or an object which is mutable. State contains all informations which are passed through our graph. Think of *State* like the global context . State params are optional. In other words, they can either hold some value or not.

**Let's create a state class**

In [33]:
# State class
class AuthState(TypedDict):
    username:Optional[str]
    password:Optional[str]
    isAuthenticated:Optional[bool]

In [34]:
UserOne:AuthState = {
    "username":"barack adma",
    "password":"password",
    "isAuthenticated":False
}
UserTwo:AuthState = {
    "username":"ido",
    "password":"jiji",
    "isAuthenticated":False
}
print(UserOne)

{'username': 'barack adma', 'password': 'password', 'isAuthenticated': False}


# Nodes
It is a unit of our graph. We could even say it is the core part. A node handles some specifics or key tasks that our agent has to do. For instance , a node may retrieve some data from a database, another will sum up this informations and so on.


We can define a node which take some informations from our user

In [35]:
def inputNode(state):
    if state["username"] == "": # you could also do state.get("username",") == ""
        state["username"] = input("set your username")

    password= input("Set your password")
    return {"password":password}

inputNode(UserOne)

Set your password pas


{'password': 'pas'}

For validating ,we can use a validating node. For instance, if temperature can't be lower than 0 , we can throw an error to ask user to set another value

In [36]:
def validationNode(state):
    username = state.get("username","")
    password= state.get("password","")

    if username == "ido" and password == "jiji":
        isAuthenticated = True
    else:
        isAuthenticated = False

    return {"isAuthenticated":isAuthenticated}
    
validationNode(UserOne)

{'isAuthenticated': False}

## After validation , we have two branch: success or fail .We must add a node to both of them

In [37]:
def successNode(state):
    return {"output":"Validation succeeded"}

def failNode(state):
    return {"output":"Validation failed! Please try again"}

successNode(UserOne)

{'output': 'Validation succeeded'}

# Router Node

In [38]:
def routerNode(state):
    if state["isAuthenticated"]:
        return "successNode"
    else:
        return "failNode"

routerNode(UserOne)

'failNode'

We can also create succes or failure node ...
# Lets see how to add node to graph
We use  **StateGraph** for achieving this

In [39]:
workflow  = StateGraph(AuthState)
workflow

For adding a node to a graph , we must set:

*   node name
*   node function

In [40]:
workflow.add_node("inputNode",inputNode)

In [41]:
workflow.add_node("validationNode",validationNode)

In [42]:
workflow.add_node("success",successNode)

In [43]:
workflow.add_node("fail",failNode)

We can't define a node twice. It will throw an err. For instance if you write this lien:workflow.add_node("input_node",inputNode)
you'll get an error like :'input_node' is defined

**Edge**:Like it sounds, edge is the connection between nodes . It leads the agents during its transition from one node to another. To add an edge, it is quiete simple: we write workflow.add_edge("pastNode","nextNode")

In [44]:
workflow.add_edge("inputNode","validationNode")

To conclude the flow after agents has succeeded a task , we can put an edge between success node and END

In [45]:
workflow.add_edge("success", END)

In [46]:
workflow.add_edge("fail", "inputNode")

For enabling decision-making , we can use conditional_edges:It is like if else.
We must first and foremost define routerNode.
Params of add_conditional_edges:


*   start  
*   routerNode

*  conditions:Regarding to the condition state , we will target one the two nodes





In [47]:
workflow.add_conditional_edges("validationNode", routerNode, {
    "successNode": "success",
    "failNode": "fail"
})

**Entry point**:It is where we must start our graph

In [48]:
workflow.set_entry_point("inputNode")

In [49]:
app = workflow.compile()

Lets invoke our app with UserOne and see the result

In [50]:
result = app.invoke(UserTwo)
print(result)

Set your password jiji


{'username': 'ido', 'password': 'jiji', 'isAuthenticated': True}


Our Authentication succeeded. Actually , our authentication is too simple. My purpose lies on explaining what langgraph is , not validation process:-)

# 2)Questions/Answer with chat ollama

In [52]:
class QaState(TypedDict):
    question:Optional[str]
    context:Optional[Tuple[str,...]]
    answer:Optional[str]

QaStateOne:QaState = {
    "question":"What is Agriconnect",
    "context":"Agriconnect level up AI application in agriculture. Allow farmers to get suitable informations",
    "answer":None
}
""" We could also do
qa_state_example = QAState(
    question="What is the purpose of this guided project?",
    context="Agriconnect level up AI application in agriculture. Allow farmers to get suitable informations",
    answer=None
)
"""

# Lets print attributes
for key, value in QaStateOne.items():
    print(f"{key}: {value}")

question: What is Agriconnect
context: Agriconnect level up AI application in agriculture. Allow farmers to get suitable informations
answer: None


# InputValidationNode

In [53]:
def InputValidationNode(state):
    question = state.get("question","")

    if not question:
        state["question"] = input("Enter your questio,")
        
    return state

InputValidationNode(QaStateOne)



{'question': 'What is Agriconnect',
 'context': 'Agriconnect level up AI application in agriculture. Allow farmers to get suitable informations',
 'answer': None}

# Context Node
It allows us to get the context of the question

In [54]:
def contextNode(state):
    question = state.get("question", "").lower()
        # We use a simple context retriever.
    if "agriconnect" in question or "agri" in question:
        # For the context we use a tuple, cause context can be retrieve from multiple documents
        context = (
            "Agriconnect level up AI application in agriculture. Allow farmers to get suitable informations",
           # "This guided project is about using LangGraph, a Python library to design state-based workflows for agriculture ",
           # "LangGraph simplifies building complex applications by connecting modular nodes with conditional edges.",
        )
        state["context"] = context
        return {"context": context}

    return {"context": None}

contextNode(QaStateOne)

{'context': ('Agriconnect level up AI application in agriculture. Allow farmers to get suitable informations',)}

In [55]:
QaStateOne

{'question': 'What is Agriconnect',
 'context': ('Agriconnect level up AI application in agriculture. Allow farmers to get suitable informations',),
 'answer': None}

# Adding llm
Lets add a node for llm part

In [56]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

In [57]:
llm = ChatOllama(model="gemma:2b")

In [58]:
# Exemple of using llm for a QA
llm.invoke("who are you")

AIMessage(content='I am a large language model, trained by Google. I am a conversational AI that can help with a wide range of tasks, such as answering questions, generating text, and translating languages.', additional_kwargs={}, response_metadata={'model': 'gemma:2b', 'created_at': '2025-11-06T19:22:15.1166333Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9657255700, 'load_duration': 3778344800, 'prompt_eval_count': 25, 'prompt_eval_duration': 1178585400, 'eval_count': 39, 'eval_duration': 4660558100, 'model_name': 'gemma:2b', 'model_provider': 'ollama'}, id='lc_run--89c75c7f-bdb2-4152-9368-7b026bce34cc-0', usage_metadata={'input_tokens': 25, 'output_tokens': 39, 'total_tokens': 64})

In [ ]:
def qaNode(state):
    question  = state.get("question","")
    context = state.get("context","")

    if not context:
        return {"answer": "I don't have enough context to answer your question."}

    Template = f"Context: {context}\nQuestion: {question}\nAnswer the question based on the provided context.Explain with knowledge.User must understand clearly"
    prompt = PromptTemplate(
        input_variables = ['context','question'],
        template=Template
    )

    formatedPrompt = prompt.format(context=context,question=question)
    try:
        response = llm.invoke(formatedPrompt)
        state["answer"] = response.content
        return {"answer":response.content}
    except Exception as e:
        return {"error":f"We get this error:{e}"}

qaNode(QaStateOne)

# Workflow

In [ ]:
qaWorkflow = StateGraph(QaState)


In [ ]:
# Adding nodes
qaWorkflow.add_node("inputNode", InputValidationNode)
qaWorkflow.add_node("contextNode", contextNode)
qaWorkflow.add_node("qANode", qaNode)


In [ ]:
qaWorkflow.set_entry_point("inputNode")

# Add edges

In [ ]:
qaWorkflow.add_edge("inputNode", "contextNode")
qaWorkflow.add_edge("contextNode", "qANode")
qaWorkflow.add_edge("qANode", END)



In [ ]:
qaApp = qaWorkflow.compile()

qaApp.invoke(QaStateOne)